# KLUE Relation Extraction - 데이터셋 구성

이 노트북은 EDA 결과를 바탕으로 **모델 학습에 바로 사용할 입력 포맷**과 **전처리 파이프라인**을 설계·구성하는 것을 목표로 합니다.

## 1. 환경 설정

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import ast
from collections import Counter

SEED = 42
np.random.seed(SEED)
print(f"✅ Seed fixed: {SEED}")

✅ Seed fixed: 42


In [2]:
# 결과 저장 디렉터리 설정
from pathlib import Path
import json

NOTEBOOK_NAME = "klue_re_dataset_construction"
RESULT_DIR = Path("temp_result") / NOTEBOOK_NAME
RESULT_DIR.mkdir(parents=True, exist_ok=True)


def _to_builtin(obj):
    if isinstance(obj, dict):
        converted = {}
        for k, v in obj.items():
            kk = _to_builtin(k)
            # JSON keys must be basic types; fallback to string
            if isinstance(kk, (dict, list, tuple)):
                kk = str(kk)
            converted[kk] = _to_builtin(v)
        return converted
    if isinstance(obj, (list, tuple)):
        return [_to_builtin(v) for v in obj]
    try:
        return obj.item()
    except Exception:
        return obj


def save_json(filename, obj):
    path = RESULT_DIR / filename
    path.write_text(json.dumps(_to_builtin(obj), ensure_ascii=False, indent=2), encoding="utf-8")
    return path


def save_text(filename, text):
    path = RESULT_DIR / filename
    path.write_text(text, encoding="utf-8")
    return path


def save_csv(filename, df):
    path = RESULT_DIR / filename
    df.to_csv(path, index=False)
    return path

print(f"✅ 결과 저장 경로: {RESULT_DIR}")

✅ 결과 저장 경로: temp_result/klue_re_dataset_construction


## 2. 데이터 로드

In [3]:
train_df = pd.read_csv('data/klue_re_train.csv')
valid_df = pd.read_csv('data/klue_re_validation.csv')

print(f"Train: {len(train_df):,} / Valid: {len(valid_df):,}")
train_df.head()

Train: 32,470 / Valid: 7,765


,guid,sentence,subject_entity,object_entity,label,source
0,klue-re-v1_train_00000,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",0,wikipedia
1,klue-re-v1_train_00001,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",0,wikitree
2,klue-re-v1_train_00002,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",5,wikitree
3,klue-re-v1_train_00003,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",10,wikitree
4,klue-re-v1_train_00004,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",0,wikipedia


## 3. 스키마 확인

In [4]:
print("컬럼:", train_df.columns.tolist())
print(train_df.sample(3, random_state=SEED)[['sentence','subject_entity','object_entity','label','source']])

컬럼: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source']
                                                sentence  \
2893            문하시중 이자연의 조카이고 인예왕후, 인경현비, 인절현비와는 사촌간이다.   
25844  1년 후, 바이에른은 전설적인 오스트리아인 감독 에른스트 하펠이 이끄는 함부르크 S...   
17759  그러다가 1949년 6월 6일 이승만 대통령과 신성모 내무부 장관의 사주를 받은 친...   

                                          subject_entity  \
2893   {'word': '인절현비', 'start_idx': 27, 'end_idx': 3...   
25844  {'word': '에른스트 하펠', 'start_idx': 27, 'end_idx'...   
17759  {'word': '신성모', 'start_idx': 26, 'end_idx': 28...   

                                           object_entity  label     source  
2893   {'word': '이자연', 'start_idx': 5, 'end_idx': 7, ...     25  wikipedia  
25844  {'word': '오스트리아', 'start_idx': 17, 'end_idx': ...     17  wikipedia  
17759  {'word': '이승만', 'start_idx': 17, 'end_idx': 19...     26  wikipedia  


In [5]:
# 결과 저장: 데이터 요약
save_json("data_overview.json", {
    "train_shape": train_df.shape,
    "valid_shape": valid_df.shape,
    "columns": train_df.columns.tolist(),
})

# 샘플 저장
sample_df = train_df.sample(3, random_state=SEED)[['sentence','subject_entity','object_entity','label','source']]
save_csv("train_sample.csv", sample_df)

PosixPath('temp_result/klue_re_dataset_construction/train_sample.csv')

## 4. 엔티티 파싱 및 정합성 검증
- `subject_entity`, `object_entity`는 문자열 형태의 dict
- start/end span이 문장과 일치하는지 확인

In [6]:
# 엔티티 파싱

def parse_entity(entity_str):
    return ast.literal_eval(entity_str)

train_df['subject_parsed'] = train_df['subject_entity'].map(parse_entity)
train_df['object_parsed'] = train_df['object_entity'].map(parse_entity)
valid_df['subject_parsed'] = valid_df['subject_entity'].map(parse_entity)
valid_df['object_parsed'] = valid_df['object_entity'].map(parse_entity)

# 스팬 정합성 검사

def span_matches(row):
    sent = row['sentence']
    s = row['subject_parsed']
    o = row['object_parsed']
    if sent[s['start_idx']:s['end_idx']+1] != s['word']:
        return False
    if sent[o['start_idx']:o['end_idx']+1] != o['word']:
        return False
    return True

train_ok = train_df.apply(span_matches, axis=1).mean()
valid_ok = valid_df.apply(span_matches, axis=1).mean()
print(f"Train span match: {train_ok:.4f}")
print(f"Valid span match: {valid_ok:.4f}")

Train span match: 1.0000
Valid span match: 1.0000


In [7]:
# 결과 저장: 엔티티 스팬 정합성
save_json("entity_span_check.json", {
    "train_span_match": train_ok,
    "valid_span_match": valid_ok,
})

PosixPath('temp_result/klue_re_dataset_construction/entity_span_check.json')

## 5. 입력 포맷 설계 (후보 비교)
EDA 결과를 고려해 **엔티티 위치를 명확히 표시하는 포맷**이 유리합니다.

### 후보 예시
- **Raw**: 원문 그대로
- **Marker**: 엔티티 시작/끝 토큰 삽입
- **Marker + Type**: 엔티티 타입까지 명시

In [8]:
# 포맷 후보 함수

def build_raw(row):
    return row['sentence']

def build_marker(row):
    sent = row['sentence']
    s = row['subject_parsed']
    o = row['object_parsed']

    # 뒤에서부터 처리 (인덱스 꼬임 방지)
    entities = [
        (s['start_idx'], s['end_idx'], '[SUBJ]', '[/SUBJ]'),
        (o['start_idx'], o['end_idx'], '[OBJ]', '[/OBJ]'),
    ]
    entities.sort(key=lambda x: x[0], reverse=True)

    result = sent
    for start, end, l, r in entities:
        result = result[:start] + l + result[start:end+1] + r + result[end+1:]
    return result

# Marker+Type 포맷은 아래 셀에서 결정한 규칙에 맞게 구현

### 예시 출력

In [9]:
sample = train_df.iloc[0]
print("RAW:", build_raw(sample))
print("MARKER:", build_marker(sample))

RAW: 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
MARKER: 〈Something〉는 [OBJ]조지 해리슨[/OBJ]이 쓰고 [SUBJ]비틀즈[/SUBJ]가 1969년 앨범 《Abbey Road》에 담은 노래다.


In [10]:
# 결과 저장: 포맷 예시
save_json("format_examples.json", {
    "raw": build_raw(sample),
    "marker": build_marker(sample),
})

PosixPath('temp_result/klue_re_dataset_construction/format_examples.json')

## 6. 전처리 파이프라인 (선택 포맷 적용)
여기서는 예시로 **Marker 포맷**을 선택합니다.

In [11]:
# 선택 포맷
train_df['input_text'] = train_df.apply(build_marker, axis=1)
valid_df['input_text'] = valid_df.apply(build_marker, axis=1)

print(train_df[['input_text','label']].head())

                                          input_text  label
0  〈Something〉는 [OBJ]조지 해리슨[/OBJ]이 쓰고 [SUBJ]비틀즈[/...      0
1  호남이 기반인 바른미래당·[OBJ]대안신당[/OBJ]·[SUBJ]민주평화당[/SUB...      0
2  K리그2에서 성적 1위를 달리고 있는 [SUBJ]광주FC[/SUBJ]는 지난 26일...      5
3  균일가 생활용품점 (주)[SUBJ]아성다이소[/SUBJ](대표 [OBJ]박정부[/O...     10
4  [OBJ]1967[/OBJ]년 프로 야구 드래프트 1순위로 [SUBJ]요미우리 자이...      0


In [12]:
# 결과 저장: 전처리 결과 샘플
save_csv("processed_head.csv", train_df[['guid','input_text','label']].head(20))

PosixPath('temp_result/klue_re_dataset_construction/processed_head.csv')

## 7. 라벨 매핑
모델 학습용 ID 매핑

In [13]:
labels = sorted(train_df['label'].unique())
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

print(label2id)

{np.int64(0): 0, np.int64(1): 1, np.int64(2): 2, np.int64(3): 3, np.int64(4): 4, np.int64(5): 5, np.int64(6): 6, np.int64(7): 7, np.int64(8): 8, np.int64(9): 9, np.int64(10): 10, np.int64(11): 11, np.int64(12): 12, np.int64(13): 13, np.int64(14): 14, np.int64(15): 15, np.int64(16): 16, np.int64(17): 17, np.int64(18): 18, np.int64(19): 19, np.int64(20): 20, np.int64(21): 21, np.int64(22): 22, np.int64(23): 23, np.int64(24): 24, np.int64(25): 25, np.int64(26): 26, np.int64(27): 27, np.int64(28): 28, np.int64(29): 29}


In [14]:
# 결과 저장: 라벨 매핑
save_json("label_mapping.json", {
    "label2id": label2id,
    "id2label": id2label,
})

PosixPath('temp_result/klue_re_dataset_construction/label_mapping.json')

## 8. 품질 점검
- 결측/중복 확인
- 변환 후 텍스트 길이 확인

In [15]:
print("결측치:")
print(train_df[['input_text','label']].isnull().sum())

print("중복 (sentence+subject+object):", train_df.duplicated(subset=['sentence','subject_entity','object_entity']).sum())

train_df['input_len'] = train_df['input_text'].str.len()
print("입력 길이 요약:")
print(train_df['input_len'].describe())

결측치:
input_text    0
label         0
dtype: int64
중복 (sentence+subject+object): 47
입력 길이 요약:
count    32470.000000
mean       121.083954
std         47.939902
min         38.000000
25%         88.000000
50%        111.000000
75%        142.000000
max        479.000000
Name: input_len, dtype: float64


In [16]:
# 결과 저장: 품질 점검 요약
missing = train_df[['input_text','label']].isnull().sum().to_dict()
dup_count = int(train_df.duplicated(subset=['sentence','subject_entity','object_entity']).sum())
length_desc = train_df['input_len'].describe().to_dict()

save_json("quality_checks.json", {
    "missing": missing,
    "duplicate_sentence_entity": dup_count,
    "input_length_desc": length_desc,
})

PosixPath('temp_result/klue_re_dataset_construction/quality_checks.json')

## 9. 저장

In [17]:
# 저장 (필요 시 실행)
try:
    train_df[['guid','input_text','label']].to_csv('data/klue_re_train_processed.csv', index=False)
    valid_df[['guid','input_text','label']].to_csv('data/klue_re_valid_processed.csv', index=False)
    print("저장 완료")
except Exception as e:
    print(f"저장 중 오류 발생: {e}")

저장 완료


## 10. 요약

- **데이터 구조 확인**: Train 32,470 / Valid 7,765 샘플, 컬럼은 `guid, sentence, subject_entity, object_entity, label, source`로 구성됨. 모델 입력에 필요한 핵심 정보가 모두 포함됨.
- **엔티티 스팬 정합성**: subject/object span이 문장과 Train/Valid 모두 100% 일치하여 전처리 오류 가능성이 낮음.
- **입력 포맷 결정**: raw 대비 **Marker 포맷**이 엔티티 위치와 역할을 명확히 전달해 RE에 유리하다고 판단.
- **전처리 품질**: 결측치 0건, 중복(문장+엔티티 기준) 47건 존재. 필요 시 중복 제거/가중치 조정 고려.
- **입력 길이 특성**: 마커 삽입 후 평균 121자, 최대 479자. 일부 긴 샘플에 대한 truncation 전략 필요.
- **라벨 매핑 확정**: 라벨 0~29 매핑 완료, 학습/평가 단계에서 동일 매핑 유지 가능.
